<a href="https://colab.research.google.com/github/mtzig/LIDC_GDRO/blob/main/notebooks/LIDC_cnn_feature_subgroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!git clone https://github.com/mtzig/LIDC_GDRO.git
%cd /content/LIDC_GDRO
# %cd ..

Cloning into 'LIDC_GDRO'...
remote: Enumerating objects: 3443, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 3443 (delta 4), reused 9 (delta 2), pack-reused 3428
Receiving objects: 100% (3443/3443), 55.90 MiB | 19.90 MiB/s, done.
Resolving deltas: 100% (3152/3152), done.
Checking out files: 100% (5390/5390), done.
/content/LIDC_GDRO


In [12]:
# import os
# import torch
import pandas as pd
# import numpy as np
from dataloaders import InfiniteDataLoader
from datasets import NoduleDataset
from models import TransferModel
from loss import ERMLoss#, GDROLossAlt, GDROLoss
from train import train, test
# from torch.optim.lr_scheduler import ReduceLROnPlateau

# import torchvision
import torch
from image_data_utils import getImages

In [4]:
if torch.cuda.is_available():
    print("Good to go!")
    DEVICE = torch.device("cuda")
else:
    print("Using cpu")
    DEVICE = torch.device("cpu")

Good to go!


#To derive an upper bound on performance, we first do clustering on entire dataset

In [9]:
nodule_id, img_data = getImages(split=False, sublabels=False, binary=False, device=DEVICE)

In [6]:
dataset = NoduleDataset(*img_data)
loader = InfiniteDataLoader(dataset, batch_size=128)

In [7]:
model = TransferModel(freeze=False, binary=False, device=DEVICE)
loss_fn = ERMLoss(model,torch.nn.CrossEntropyLoss(),{})
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.005)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [8]:
epochs = 20
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train(loader, model, loss_fn, optimizer, verbose=True)

Epoch 1/15
Average training loss: 1.6096623810854824
Epoch 2/15
Average training loss: 1.0272845354947178
Epoch 3/15
Average training loss: 0.7714252851226113
Epoch 4/15
Average training loss: 0.6268660453232852
Epoch 5/15
Average training loss: 0.5220643281936646
Epoch 6/15
Average training loss: 0.43945582888343115
Epoch 7/15
Average training loss: 0.3556979461149736
Epoch 8/15
Average training loss: 0.3527693463997407
Epoch 9/15
Average training loss: 0.2775972959670154
Epoch 10/15
Average training loss: 0.25271046974442224
Epoch 11/15
Average training loss: 0.24750975451686166
Epoch 12/15
Average training loss: 0.22280371324582535
Epoch 13/15
Average training loss: 0.18296518515456806
Epoch 14/15
Average training loss: 0.18865948915481567
Epoch 15/15
Average training loss: 0.19465707716616718


# Now we extract the features

In [10]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook
model.model.avgpool.register_forward_hook(get_activation('avgpool'))
image_tensors = torch.stack(img_data[0])
model(image_tensors)
image_features = activation['avgpool'].squeeze()


In [13]:
cols = []
for idx,id in enumerate(nodule_id):
  cols.append([id] + image_features[idx].cpu().numpy().tolist())
lidc_cnnFeatures = pd.DataFrame(cols).rename({0:'nodule_id'}, axis=1)
lidc_cnnFeatures.head()

,nodule_id,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,512
0,2691,0.552001,0.007099,0.099501,0.420685,1.410234,0.030461,0.228956,0.387511,0.108710,...,2.109688,0.147571,0.038860,0.708355,0.550058,0.709375,0.160741,0.287429,1.021720,0.234102
1,2185,1.547561,0.000000,0.000000,0.889399,2.486399,0.000000,0.676981,0.113101,0.145823,...,2.294022,0.012451,0.848783,1.158062,0.853212,0.118796,0.004371,1.117693,3.343482,0.000000
2,2190,1.808401,0.011915,0.100547,0.718175,2.816071,0.037634,0.000000,0.000000,0.000000,...,2.366264,0.799264,0.706379,0.070460,0.000000,1.155147,0.094662,1.005571,0.705891,0.003502
3,990,3.043851,0.272301,0.017524,1.063373,3.085476,0.004206,0.379519,0.139327,0.000000,...,2.228153,0.452996,0.655810,0.389889,0.000881,0.119418,0.222527,0.761036,1.406825,0.000000
4,386,2.578759,0.000000,0.000000,1.653559,4.426379,0.115955,2.132362,0.547618,1.005010,...,1.326507,0.638974,0.021883,0.342488,0.661192,1.192846,0.724254,1.449755,2.335729,0.000310


In [ ]:
lidc_cnnFeatures.to_csv('cnn_features.csv')
torch.save(model.state_dict(), 'cnn_feature_model') #saves weights in file called weights

## Now we perform clustering